# ETL tutorial

In [ ]:
!pip install --upgrade google-cloud-bigquery

In [ ]:
!pip install pandas-gbq

In [ ]:
!pip install db-dtypes

In [ ]:
import pymongo

import pandas as pd
import matplotlib.pyplot as plt

from google.cloud import bigquery
from google.oauth2 import service_account

### Extracting data

In [ ]:
DB_NAME = "sample_airbnb"
COLLECTION = "listingsAndReviews"

In [ ]:
client = pymongo.MongoClient("mongodb+srv://fabiancpl:<password>@cluster0.ys2ybhm.mongodb.net/?retryWrites=true&w=majority", server_api = pymongo.server_api.ServerApi('1'))

In [ ]:
db = client[DB_NAME]

In [ ]:
col = db[COLLECTION]

In [ ]:
result = col.find({}, {"_id": 0, "address.country": 1, "address.market": 1, "price": 1})

In [ ]:
df = pd.json_normalize(result).rename(columns = {"address.country": "country", "address.market": "market"})

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.sample(5)

### Exploring and cleaning data

In [ ]:
df["country"].value_counts(dropna = False)

In [ ]:
df.loc[df["market"] == "", "market"] = "Other"

In [ ]:
df["market"].value_counts(dropna = False)

In [ ]:
df["price"] = df["price"].astype(str).astype(float)

In [ ]:
plt.figure(figsize = (20, 3))
plt.boxplot(df["price"], vert = False)
plt.show()

### Transforming data

In [ ]:
df["count"] = 1

In [ ]:
df_grouped = df.groupby(["country", "market"]).agg({"count": "count", "price": "median"}).reset_index().rename(columns = {"price": "price_median"})

In [ ]:
df_grouped.sample(5)

### Loading data

In [ ]:
credentials = service_account.Credentials.from_service_account_file("./javeriana-dataprep.json", scopes = ["https://www.googleapis.com/auth/cloud-platform"])

In [ ]:
client = bigquery.Client(credentials = credentials, project = credentials.project_id)

In [ ]:
# Creating the job config
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField("country", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("market", bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField("count", bigquery.enums.SqlTypeNames.INT64),
        bigquery.SchemaField("price_median", bigquery.enums.SqlTypeNames.FLOAT64)
    ],
    # Drod and re-create table, if exist
    write_disposition = "WRITE_TRUNCATE",
)

In [ ]:
BQ_TABLE_NAME = "dataprep.listings_country_market"

In [ ]:
# Sending the job to BigQuery
job = client.load_table_from_dataframe(
    df_grouped, BQ_TABLE_NAME, job_config = job_config
)

job.result()

In [ ]:
# Verifying if table was successfully created or updated
table = client.get_table(BQ_TABLE_NAME)

print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), BQ_TABLE_NAME))

In [ ]:
query = """SELECT * FROM `javeriana-dataprep.dataprep.listings_country_market`"""

pd.read_gbq(query, credentials = credentials)